Initial plan (or, to be buzzword-compliant, design-doc/outline/<most appropriate buzzword here>)

1. Extract weather data for one city from the raw data file
2. Calculate day of year from the year-month-day value
3. Keep hourly data for what is, approximately, not night time
4. For each hour of the day keep only the most commonly occurring weather description; discard other weather description data
5. For each day keep only the most commonly occurring weather description; discard other weather description data
6. Calculate rgb color of sky for each weather description
7. Make bar chart of sky color where each bar represent one day of the year

Note: this project is based on data from an [Open Database on Kaggle](https://www.kaggle.com/selfishgene/historical-hourly-weather-data#weather_description.csv) provided by [David Beniaguev](https://davidbeniaguev.com). Thank you David!

In [1]:
import pandas as pd

#### Extract weather data for one city from the raw data file at https://www.kaggle.com/selfishgene/historical-hourly-weather-data#weather_description.csv

In [12]:
# download 'weather_description.csv' from the link above to the same repository as your Jupyter Notebook
dataFile='weather_description.csv'
city='New York'
datetime='datetime'

In [13]:
# import hourly weather descriptions
df_WeatherDescription = pd.read_csv(dataFile, usecols=[datetime, city], parse_dates = True)
df_WeatherDescription.head()

,datetime,New York
0,2012-10-01 12:00:00,NaN
1,2012-10-01 13:00:00,few clouds
2,2012-10-01 14:00:00,few clouds
3,2012-10-01 15:00:00,few clouds
4,2012-10-01 16:00:00,few clouds


In [14]:
df_WeatherDescription.rename(columns={'New York': 'weather'}, inplace=True)
df_WeatherDescription.dropna(inplace = True)
df_WeatherDescription.head()

,datetime,weather
1,2012-10-01 13:00:00,few clouds
2,2012-10-01 14:00:00,few clouds
3,2012-10-01 15:00:00,few clouds
4,2012-10-01 16:00:00,few clouds
5,2012-10-01 17:00:00,few clouds
